# Pruning
imports

In [20]:
import os
import tensorflow as tf
import tensorflow_datasets as tfds
import tensorflow_hub as hub
import tensorflow_model_optimization as tfmot
import numpy as np
import tempfile
from huggingface_hub import from_pretrained_keras

from tensorflow.python.framework import graph_util
from tensorflow.python.framework import graph_io
from tensorflow.python.tools import optimize_for_inference_lib
from tensorflow.python.platform import gfile

from src.models_download.model_download import download_tf_od_zoo
from src.models_download.model_download import download_tf_classification_zoo

paths

In [ ]:
root_path = os.path.abspath(os.path.join(os.getcwd(), os.pardir))

In [ ]:
object_detection_models = os.path.join(root_path, "pretrained_models", "object_detection")
classification_models = os.path.join(root_path, "pretrained_models", "classification")

download models

In [ ]:
os.makedirs(object_detection_models)
download_tf_od_zoo(object_detection_models)

In [ ]:
os.makedirs(classification_models)
download_tf_classification_zoo(classification_models)

prune test

In [ ]:
frozen_graph_path = os.path.join(object_detection_models, "ssd_mobilenet_v2_320x320_coco17_tpu-8", "saved_model", "saved_model.pb")
saved_model_path = os.path.join(object_detection_models, "ssd_mobilenet_v2_320x320_coco17_tpu-8", "saved_model")

In [64]:
# model = tf.keras.applications.MobileNet(input_shape=(32,32,3), include_top=False)
model = tf.keras.applications.ResNet50(input_shape=(32,32,3), include_top=False)


cifar100 = tf.keras.datasets.cifar100
(train_images, train_labels), (test_images, test_labels) = cifar100.load_data()

train_images = tf.keras.applications.resnet50.preprocess_input(train_images)
test_images = tf.keras.applications.resnet50.preprocess_input(test_images)

94765736/94765736 [==============================] - 13s 0us/step


In [75]:
dataset, info = tfds.load('imagenette', split='train', with_info=True)
print(type(dataset))
train_dataset = dataset['train']
test_dataset = dataset['train']

def preprocess(image, label):
    image = tf.cast(image, tf.float32) / 255.0
    label = tf.one_hot(label, 1000)
    return image, label

train_dataset = train_dataset.map(preprocess)
test_dataset = test_dataset.map(preprocess)

train_dataset = train_dataset.shuffle(10000).batch(32)
test_dataset = test_dataset.batch(32)

train_images, train_labels = next(iter(train_dataset))
test_images, test_labels = next(iter(test_dataset))


train_images = tf.keras.applications.resnet50.preprocess_input(train_images)
test_images = tf.keras.applications.resnet50.preprocess_input(test_images)

<class 'tensorflow.python.data.ops.dataset_ops.PrefetchDataset'>


TypeError: 'PrefetchDataset' object is not subscriptable

In [65]:
# Train the digit classification model
model.compile(optimizer='adam',
              loss=tf.keras.losses.SparseCategoricalCrossentropy(from_logits=True),
              metrics=['accuracy'])

model.fit(
  train_images,
  train_labels,
  epochs=4,
  validation_split=0.1,
)

Epoch 1/4
1407/1407 [==============================] - 31s 20ms/step - loss: 7.2306 - accuracy: 0.0085 - val_loss: 7.3353 - val_accuracy: 0.0102
Epoch 2/4
1407/1407 [==============================] - 26s 19ms/step - loss: 6.6787 - accuracy: 0.0101 - val_loss: 6.7190 - val_accuracy: 0.0105
Epoch 3/4
1407/1407 [==============================] - 26s 18ms/step - loss: 6.1564 - accuracy: 0.0103 - val_loss: 6.3381 - val_accuracy: 0.0114
Epoch 4/4
1407/1407 [==============================] - 27s 19ms/step - loss: 5.9594 - accuracy: 0.0104 - val_loss: 5.8773 - val_accuracy: 0.0108


In [66]:
_, baseline_model_accuracy = model.evaluate(
    test_images, test_labels, verbose=0)

print('Baseline test accuracy:', baseline_model_accuracy)

Baseline test accuracy: 0.011537354439496994


In [67]:
# Compute end step to finish pruning after 2 epochs.
batch_size = 128
epochs = 2
validation_split = 0.1 # 10% of training set will be used for validation set.

num_images = train_images.shape[0] * (1 - validation_split)
end_step = np.ceil(num_images / batch_size).astype(np.int32) * epochs

# Define model for pruning.
pruning_params = {
      'pruning_schedule': tfmot.sparsity.keras.PolynomialDecay(initial_sparsity=0.50,
                                                               final_sparsity=0.80,
                                                               begin_step=0,
                                                               end_step=end_step)
}

model_for_pruning = tfmot.sparsity.keras.prune_low_magnitude(model, **pruning_params)

model_for_pruning.compile(optimizer='adam',
              loss=tf.keras.losses.SparseCategoricalCrossentropy(from_logits=True),
              metrics=['accuracy'])

model_for_pruning.summary()

Model: "resnet50"
__________________________________________________________________________________________________
 Layer (type)                   Output Shape         Param #     Connected to                     
 input_10 (InputLayer)          [(None, 32, 32, 3)]  0           []                               
                                                                                                  
 prune_low_magnitude_conv1_pad   (None, 38, 38, 3)   1           ['input_10[0][0]']               
 (PruneLowMagnitude)                                                                              
                                                                                                  
 prune_low_magnitude_conv1_conv  (None, 16, 16, 64)  18882       ['prune_low_magnitude_conv1_pad[0
  (PruneLowMagnitude)                                            ][0]']                           
                                                                                           

In [ ]:
# os.environ["TF_DISABLE_JIT"] = "1"

logdir = tempfile.mkdtemp()

callbacks = [
  tfmot.sparsity.keras.UpdatePruningStep(),
  tfmot.sparsity.keras.PruningSummaries(log_dir=logdir),
]

model_for_pruning.fit(train_images, train_labels,
                  batch_size=batch_size, epochs=epochs, validation_split=validation_split,
                  callbacks=callbacks)

In [27]:
model = tf.keras.applications.MobileNetV2()

In [28]:
(train_ds, val_ds), info = tfds.load('imagenet2012', split=['train', 'validation'], with_info=True)

In [29]:
def preprocess(data):
    image = data['image']
    label = data['label']
    # Resize image to 224x224
    image = tf.image.resize(image, (224, 224))
    # Convert image to float32
    image = tf.cast(image, tf.float32)
    # Normalize image
    image = tf.keras.applications.mobilenet_v2.preprocess_input(image)
    return image, label


# train_ds = train_ds.map(preprocess).shuffle(10000).batch(32)
train_ds = train_ds.map(preprocess).batch(32)
val_ds = val_ds.map(preprocess).batch(32)

In [30]:
model.compile(optimizer='adam',
              loss=tf.keras.losses.SparseCategoricalCrossentropy(from_logits=True),
              metrics=['accuracy'])

_, baseline_model_accuracy = model.evaluate(val_ds, verbose=0)

print('Baseline test accuracy:', baseline_model_accuracy)

/home/marcinwsl/miniconda3/envs/tf/lib/python3.9/site-packages/keras/backend.py:5582: UserWarning: "`sparse_categorical_crossentropy` received `from_logits=True`, but the `output` argument was produced by a Softmax activation and thus does not represent logits. Was this intended?
  output, from_logits = _get_logits(


Baseline test accuracy: 0.7064200043678284


In [ ]:
converter = tf.lite.TFLiteConverter.from_keras_model(model)
tflite_model = converter.convert()

with open('model.tflite', 'wb') as f:
    f.write(tflite_model)

In [31]:
# Compute end step to finish pruning after 2 epochs.
batch_size = 128
epochs = 2
validation_split = 0.1 # 10% of training set will be used for validation set.

num_images = train_ds.__len__().numpy() * (1 - validation_split)
end_step = np.ceil(num_images / batch_size).astype(np.int32) * epochs

# Define model for pruning.
pruning_params = {
      'pruning_schedule': tfmot.sparsity.keras.PolynomialDecay(initial_sparsity=0.50,
                                                               final_sparsity=0.80,
                                                               begin_step=0,
                                                               end_step=end_step)
}

model_for_pruning = tfmot.sparsity.keras.prune_low_magnitude(model, **pruning_params)

model_for_pruning.compile(
    optimizer='adam',
    loss=tf.keras.losses.SparseCategoricalCrossentropy(from_logits=True),
    metrics=['accuracy']
)

model_for_pruning.summary()

Model: "mobilenetv2_1.00_224"
__________________________________________________________________________________________________
 Layer (type)                   Output Shape         Param #     Connected to                     
 input_3 (InputLayer)           [(None, 224, 224, 3  0           []                               
                                )]                                                                
                                                                                                  
 prune_low_magnitude_Conv1 (Pru  (None, 112, 112, 32  1730       ['input_3[0][0]']                
 neLowMagnitude)                )                                                                 
                                                                                                  
 prune_low_magnitude_bn_Conv1 (  (None, 112, 112, 32  129        ['prune_low_magnitude_Conv1[0][0]
 PruneLowMagnitude)             )                                ']            

In [34]:
logdir = tempfile.mkdtemp()

callbacks = [
  tfmot.sparsity.keras.UpdatePruningStep(),
  tfmot.sparsity.keras.PruningSummaries(log_dir=logdir),
]

model_for_pruning.fit(train_ds, validation_data=val_ds,
                  batch_size=batch_size, epochs=epochs,
                  callbacks=callbacks)

Epoch 1/2


/home/marcinwsl/miniconda3/envs/tf/lib/python3.9/site-packages/keras/backend.py:5582: UserWarning: "`sparse_categorical_crossentropy` received `from_logits=True`, but the `output` argument was produced by a Softmax activation and thus does not represent logits. Was this intended?
  output, from_logits = _get_logits(
error: error: Can't find libdevice directory ${CUDA_DIR}/nvvm/libdevice
Can't find libdevice directory ${CUDA_DIR}/nvvm/libdeviceerror: 
Can't find libdevice directory ${CUDA_DIR}/nvvm/libdevice
error: Can't find libdevice directory ${CUDA_DIR}/nvvm/libdevice
error: error: Can't find libdevice directory ${CUDA_DIR}/nvvm/libdeviceCan't find libdevice directory ${CUDA_DIR}/nvvm/libdevice

error: error: Can't find libdevice directory ${CUDA_DIR}/nvvm/libdeviceCan't find libdevice directory ${CUDA_DIR}/nvvm/libdevice

error: error: Can't find libdevice directory ${CUDA_DIR}/nvvm/libdeviceCan't find libdevice directory ${CUDA_DIR}/nvvm/libdevice

error: error: Can't find libdevi

UnknownError: Graph execution error:

Detected at node 'mobilenetv2_1.00_224/prune_low_magnitude_Conv1/FloorMod' defined at (most recent call last):
    File "/home/marcinwsl/miniconda3/envs/tf/lib/python3.9/runpy.py", line 197, in _run_module_as_main
      return _run_code(code, main_globals, None,
    File "/home/marcinwsl/miniconda3/envs/tf/lib/python3.9/runpy.py", line 87, in _run_code
      exec(code, run_globals)
    File "/home/marcinwsl/miniconda3/envs/tf/lib/python3.9/site-packages/ipykernel_launcher.py", line 17, in <module>
      app.launch_new_instance()
    File "/home/marcinwsl/miniconda3/envs/tf/lib/python3.9/site-packages/traitlets/config/application.py", line 1043, in launch_instance
      app.start()
    File "/home/marcinwsl/miniconda3/envs/tf/lib/python3.9/site-packages/ipykernel/kernelapp.py", line 725, in start
      self.io_loop.start()
    File "/home/marcinwsl/miniconda3/envs/tf/lib/python3.9/site-packages/tornado/platform/asyncio.py", line 215, in start
      

In [ ]:
model = from_pretrained_keras("keras-io/Object-Detection-RetinaNet")

In [ ]:
converter = tf.lite.TFLiteConverter.from_keras_model(model)
tflite_model = converter.convert()

with open('model.tflite', 'wb') as f:
    f.write(tflite_model)

In [29]:
data, info = tfds.load('coco/2017', split='train', with_info=True)

# Prepare the dataset for training
# def prepare_example(example):
#     # Normalize the image
#     image = tf.cast(example['image'], tf.float32) / 255.
#     # Resize the image to (256, 256)
#     image = tf.image.resize(image, (256, 256))
#     # Convert the labels and bboxes to dense tensors
#     label = tf.sparse.to_dense(example['objects']['label'])
#     bbox = tf.sparse.to_dense(example['objects']['bbox'])
#     return image, {'class_output': label, 'bbox_output': bbox}

def prepare_example(example):
    # Normalize the image
    image = tf.cast(example['image'], tf.float32) / 255.
    # Resize the image to (256, 256)
    image = tf.image.resize(image, (800, 800))
    # Convert the labels and bboxes to dense tensors
    label = example['objects']['label']
    bbox = example['objects']['bbox']
    label = tf.one_hot(label, depth=info.features['objects']['label'].num_classes)
    label = tf.reduce_max(label, axis=0)
    bbox = tf.reshape(bbox, [-1, 4])
    return image, {'class_output': label, 'bbox_output': bbox}

train_data = data.map(prepare_example).batch(32)

pruned_layers = []
for layer in model.layers:
    if isinstance(layer, tf.keras.layers.Conv2D) or isinstance(layer, tf.keras.layers.Dense):
        # Apply pruning to the layer if it is a supported layer type
        pruning_params = {
            'pruning_schedule': tfmot.sparsity.keras.ConstantSparsity(0.5, begin_step=0, end_step=100, frequency=10),
            'block_size': (1, 1),
            'block_pooling_type': 'AVG'
        }
        pruned_layer = tfmot.sparsity.keras.prune_low_magnitude(layer, **pruning_params)
        pruned_layers.append(pruned_layer)
    else:
        # Add the layer to the list of unpruned layers if it is not a supported layer type
        pruned_layers.append(layer)

pruned_model = tf.keras.models.Sequential(pruned_layers)

# Compile the pruned model
pruned_model.compile(loss={'class_output': 'binary_crossentropy', 'bbox_output': 'mse'},
                      optimizer='adam', metrics=['accuracy'])

pruned_model.fit(train_data, epochs=10)

converter = tf.lite.TFLiteConverter.from_keras_model(pruned_model)
tflite_model = converter.convert()

with open('pruned_model.tflite', 'wb') as f:
    f.write(tflite_model)

Epoch 1/10


ValueError: in user code:

    File "/home/marcinwsl/miniconda3/envs/tf/lib/python3.9/site-packages/keras/engine/training.py", line 1160, in train_function  *
        return step_function(self, iterator)
    File "/home/marcinwsl/miniconda3/envs/tf/lib/python3.9/site-packages/keras/engine/training.py", line 1146, in step_function  **
        outputs = model.distribute_strategy.run(run_step, args=(data,))
    File "/home/marcinwsl/miniconda3/envs/tf/lib/python3.9/site-packages/keras/engine/training.py", line 1135, in run_step  **
        outputs = model.train_step(data)
    File "/home/marcinwsl/miniconda3/envs/tf/lib/python3.9/site-packages/keras/engine/training.py", line 993, in train_step
        y_pred = self(x, training=True)
    File "/home/marcinwsl/miniconda3/envs/tf/lib/python3.9/site-packages/keras/utils/traceback_utils.py", line 70, in error_handler
        raise e.with_traceback(filtered_tb) from None
    File "/home/marcinwsl/miniconda3/envs/tf/lib/python3.9/site-packages/k

In [30]:
# Define the pruning parameters
pruning_params = {
    'pruning_schedule': tfmot.sparsity.keras.PolynomialDecay(initial_sparsity=0.50, final_sparsity=0.90, begin_step=0, end_step=1000)
}

pruned_model = model

# Prune only the supported layers of the model
for layer in pruned_model.layers:
    if isinstance(layer, (tf.keras.layers.Conv2D, tf.keras.layers.Dense)):
        tfmot.sparsity.keras.prune_low_magnitude(layer, **pruning_params)

# Compile and train the pruned model
pruned_model.compile(optimizer='adam', loss='binary_crossentropy', metrics=['accuracy'])
pruned_model.fit(train_data, epochs=10)

Epoch 1/10


ValueError: in user code:

    File "/home/marcinwsl/miniconda3/envs/tf/lib/python3.9/site-packages/keras/engine/training.py", line 1160, in train_function  *
        return step_function(self, iterator)
    File "/home/marcinwsl/miniconda3/envs/tf/lib/python3.9/site-packages/keras/engine/training.py", line 1146, in step_function  **
        outputs = model.distribute_strategy.run(run_step, args=(data,))
    File "/home/marcinwsl/miniconda3/envs/tf/lib/python3.9/site-packages/keras/engine/training.py", line 1135, in run_step  **
        outputs = model.train_step(data)
    File "/home/marcinwsl/miniconda3/envs/tf/lib/python3.9/site-packages/keras/engine/training.py", line 994, in train_step
        loss = self.compute_loss(x, y, y_pred, sample_weight)
    File "/home/marcinwsl/miniconda3/envs/tf/lib/python3.9/site-packages/keras/engine/training.py", line 1052, in compute_loss
        return self.compiled_loss(
    File "/home/marcinwsl/miniconda3/envs/tf/lib/python3.9/site-packages/ker